In [22]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras


# Common imports
import numpy as np
import os
import random
import time
from IPython.display import clear_output


# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

n=10
m=10

from collections import deque

replay_memory = deque(maxlen=20000)


In [23]:
from google.colab import drive


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Intern code:

- 0 : out of grid and wall (x and #)
- 1 : small ball
- 2 : medium ball
- 3 : small ball on top of a medium ball
- 4 : large ball
- 5 : small ball on top of a large ball
- 6 : medium ball on top of a large ball
- 7 : small ball on top of a medium ball on top of a large ball
- 8 : grass (,)
- 9 : snow (.)
- 10: character with snow on the floor (p) (aquest no compte a la matriu d'accions)
- 11: character with grass on the floor (q) (aquest no compte a la matriu d'accions)


Reconpenses
- 0 moure's sense apretar
- 0 moure's apretant bola petita
- 0 moure's apretant bola mitjana
- 0 moure's apretant bola grossa
- 100 col.locar bola mitjana sobre bola grossa
- 500 col.locar bola petita sobre boles mitjnes i grosses
- -1 Passar un instant

Accions prohibides (-100 punts)
- sortir de la quadricula (trepitjar pared)
- fer sortir bola de la quadricula (trepitjar pared la bola)
- fer 2 boles grans
- fer dos boles mitjanes si ja tenim bola gran


Maxim episodi=50 jugades, fins acció prohibida o col.locar tres boles be

In [24]:
error=-100
tonto=-10
cami=10
cim=500
convertir=100
bingo=2000

actions=[
    [[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error]],
    [[None,None,None,error],[None,None,None,tonto],['pq',11,3,cim],[None,None,None,tonto],['pq',11,5,cim],[None,None,None,tonto],['pq',11,7,bingo],[None,None,None,tonto],['pq',11,1,cami],['pq',11,2,convertir]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,6,cim],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,2,cami],['pq',11,4,convertir]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,2,1,-cim],[None,2,2,-cim]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,4,cami],['pq',11,4,cami]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,4,1,-cim],[None,4,2,-cim]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,4,2,-cim],[None,4,4,-cim]],
    [[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error]],
    [['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,tonto],['pq',11,None,cami],['pq',11,None,cami]],
    [['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,tonto],['pq',10,None,cami],['pq',10,None,cami]]
]

'''
Es tracta d'una matriu de 10x10 la primera dimensió es que hi ha a la posició seguent al jugador (home) i la segona que hi ha a la posició seguent de la seguent del jugador
segons l'accio que hem agafat (action = 0 dreta, 1 baix, 2 esquerra, 3 dalt)
Es de 10x10 ja que, el 10 i el 11 es impossible que estiguin a la posició seguent i seguent de la sefgent, ja que tant sols tenim un jugador i aquest esta en la
posició actual
La tercera dimensió de la matriu significa que cal fer amb la posició del jugador, la posició seguent del jugador, la posició següent de la seguent i la recompensa.
pq, significa que si el jugador actual esta sobre snow, o sigui un 10, al avancar hem de torna a possar un 10 i si estar sobre grass, o sigui un 11, al avançar hem de tornar a possar un 11
'''


"\nEs tracta d'una matriu de 10x10 la primera dimensió es que hi ha a la posició seguent al jugador (home) i la segona que hi ha a la posició seguent de la seguent del jugador\nsegons l'accio que hem agafat (action = 0 dreta, 1 baix, 2 esquerra, 3 dalt)\nEs de 10x10 ja que, el 10 i el 11 es impossible que estiguin a la posició seguent i seguent de la sefgent, ja que tant sols tenim un jugador i aquest esta en la\nposició actual\nLa tercera dimensió de la matriu significa que cal fer amb la posició del jugador, la posició seguent del jugador, la posició següent de la seguent i la recompensa.\npq, significa que si el jugador actual esta sobre snow, o sigui un 10, al avancar hem de torna a possar un 10 i si estar sobre grass, o sigui un 11, al avançar hem de tornar a possar un 11\n"

In [25]:
ree='x#,.qp1234567'
reem=[0,0,8,9,11,10,1,2,3,4,5,6,7]
def lleguir_tauler(nom='C:/Users/josep/Snowman/Dades/suy2.txt',n=n,m=m):
    f = open(nom, "r")
    tauler=np.zeros((n,m))
    for row,linea in enumerate(f):
        linea=linea.rstrip('\n\r\t')
        for column,car in enumerate(linea):
            res = ree.find(car)
            tauler[row,column]=reem[res]
    f.close()
    return tauler

n_outputs=4


In [26]:
def step(t,action): # action = 0 dreta, 1 baix, 2 esquerra, 3 dalt
    snow=True
    res = np.where(t == 10) #si el jugador esta sobre snow
    if np.size(res)==0:
        snow=False
        res = np.where(t == 11) #si el jugador esta grass
    #posició del jugador t[a,b]

    a=res[0]
    b=res[1]


    inc=[[0,1,0,-1],[1,0,-1,0]]
    seg=[a+inc[0][action],b+inc[1][action]] #seguent posició segons l'acció a realitzar
    seg2=[a+2*inc[0][action],b+2*inc[1][action]] #seguent de la seguent posició segons l'acció a realitzar

    #print(seg,seg2,t.shape)

    mov=actions[int(t[seg[0],seg[1]])][int(t[seg2[0],seg2[1]])] #busca a la matri d'acions segons el que hi ha a la posició seguent i la seguent de la seguent
    '''print(int(t[seg[0],seg[1]]),int(t[seg2[0],seg2[1]]))
    print(mov)
    aa=input()'''
    reward=mov[3] # la tercera psosició es la recompensa
    mov=mov[:3] # les tres primeres posicions son que hem de col.locar a la posició del jugador, la posició seguent i la posició seguent de la seguent

    for i,aux in enumerate(mov):
        if aux!=None:
            if aux=='pq':
                if snow:
                    f=9
                else:
                    f=8
            else:
                f=int(aux)
            #print('f:',f)
            if i==0:
                t[a,b]=f
            elif i==1:
                t[seg[0],seg[1]]=f
            else:
                t[seg2[0],seg2[1]]=f
    if reward==bingo or reward<=tonto:
        done=True
    else:
        done=False

    return t,reward,done




In [27]:
'''def arreglar_tauler(t): # Passo cada posicio del tauler a l'interval [0,1]
    t2=t.copy()
    res = np.where(t2 == 11)
    if np.size(res)!=0:
        t2[res[0],res[1]]=10
    t2=t2/10.
    return t2

# Aquesta funció s'ha de modificar i que doni un taula de 12 de profunditat'''



transform=[[0],[4],[5],[4,5],[6],[4,6],[5,6],[4,5,6],[2],[1],[1,3],[2,3]]

def arreglar_tauler(t): #dimensions: 0 pared, 1 neu, 2 herba, 3 jugador, 4 bola petita, 5 bola mitjana, 6 bola gran
    tauler=np.zeros((n, m, 7))
    for i in range(n):
      for j in range(m):
        for k in transform[int(t[i][j])]:
          tauler[i,j,k]=1
    return tauler


In [28]:
def epsilon_greedy_policy(t, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)
    else:
        Q_values = model.predict(arreglar_tauler(t)[np.newaxis],verbose=0)
        return np.argmax(Q_values[0])


In [29]:
tf.keras.backend.clear_session()
np.random.seed(13)
tf.random.set_seed(13)


model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8,(3,3),activation='tanh',padding='same',input_shape=(n,m,7)),
    #tf.keras.layers.Conv2D(32,(3,3),activation='tanh',padding='same'),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dense(128,activation='tanh'),
    tf.keras.layers.Dense(16,activation='tanh'),
    tf.keras.layers.Dense(4,activation='softmax')
])
print(model.summary())

target = keras.models.clone_model(model)
target.set_weights(model.get_weights())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 10, 10, 8)         512       
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 16)                12816     
                                                                 
 dense_1 (Dense)             (None, 4)                 68        
                                                                 
Total params: 13396 (52.33 KB)
Trainable params: 13396 (52.33 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [30]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_memory), size=batch_size)
    batch = [replay_memory[index] for index in indices]
    states, actions, rewards, next_states, dones = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(5)]
    return states, actions, rewards, next_states, dones

In [31]:
def play_one_step(t, epsilon):
    action = epsilon_greedy_policy(t, epsilon)
    next_t, reward, done= step(t,tf.constant(action))
    replay_memory.append((arreglar_tauler(t), action, reward, arreglar_tauler(next_t), done))
    return next_t, reward, done




In [32]:
batch_size = 256
discount_rate = 0.95
optimizer = keras.optimizers.Adam(learning_rate=1e-2)
loss_fn = tf.keras.losses.CategoricalCrossentropy ()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones = experiences
    next_Q_values = model.predict(next_states,verbose=0)
    a=input()
    best_next_actions = np.argmax(next_Q_values, axis=1)
    next_mask = tf.one_hot(best_next_actions, n_outputs).numpy()
    next_best_Q_values = (target.predict(next_states,verbose=0) * next_mask).sum(axis=1)
    target_Q_values = (rewards +
                       (1 - dones) * discount_rate * next_best_Q_values)
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [33]:
experiences = sample_experiences(batch_size)
print(len(experiences[0][0]),batch_size)
print(experiences[0][0])

'experiences = sample_experiences(batch_size)\nprint(len(experiences[0][0]),batch_size)\nprint(experiences[0][0])'

In [34]:
def prova():
    tauler=lleguir_tauler("/content/drive/MyDrive/Colab Notebooks/Snowman/Dades/david.txt",n=n,m=m)
    print(tauler)
    time.sleep(5)
    clear_output(wait=True)
    sum_rewards=0

    for i in range(50):
        aux=model.predict(arreglar_tauler(tauler)[np.newaxis,:,:],verbose=0).squeeze()

        action=np.argmax(aux)
        clear_output(wait=True)
        print(aux)
        tauler, reward, done = step(tauler,action)
        sum_rewards+=reward
        print(action,i,sum_rewards)
        print(tauler)
        time.sleep(5)
    #clear_output(wait=True)

In [35]:
import glob

path="/content/drive/MyDrive/Colab Notebooks/Snowman/Dades/facils/"

files=glob.glob(path+"*.txt")

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

rewards = []
best_score = -100000
episodes=150000
jugades=30


tf.keras.backend.clear_session()
model=tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Snowman/dddqn1.h5")
target.set_weights(model.get_weights())


total=0
maxim=-10000
nom="/content/drive/MyDrive/Colab Notebooks/Snowman/Dades/david.txt"
taux=lleguir_tauler(nom)
for episode in range(episodes):
    #nom=files[episode % len(files)]
    t=taux.copy()
    epsilon = max(0.8 - episode / 25000, 0.1)
    act_rewards=[]

    for st in range(jugades):
        t, reward, done  = play_one_step(t, epsilon)
        act_rewards.append(reward)
        if reward>maxim:
          maxim=reward
        if done:
            break
    score=sum(act_rewards)
    total=total+score
    '''rewards.append(score) # Not shown in the book'''
    if score >= best_score: # Not shown
        #best_weights = model.get_weights() # Not shown
        best_score = score # Not shown'''
    if episode%100==0:
        print("\rEpisode: {}, score: {}, best_score: {}, eps: {:.3f}, maxim: {} ".format(episode, total/100, best_score, epsilon,maxim), end="") # Not shown
        total=0
        maxim=-10000
    if episode >= 50:
        training_step(batch_size)
        if episode % 50 == 0:
            target.set_weights(model.get_weights())

#model.set_weights(best_weights)

<ipython-input-26-8fc40d83b7f4>:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mov=actions[int(t[seg[0],seg[1]])][int(t[seg2[0],seg2[1]])] #busca a la matri d'acions segons el que hi ha a la posició seguent i la seguent de la seguent


Episode: 0, score: -0.1, best_score: -10, eps: 0.800, maxim: 10 

In [44]:
print(taux)
print(ttt[5])

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  9.  9.  9.  9.  9.  9.  9.  0.  0.]
 [ 0.  9.  9.  9.  9.  9.  1.  9.  0.  0.]
 [ 0.  9.  1.  9.  9.  9.  9.  9.  0.  0.]
 [ 0.  9.  9.  9.  9.  9.  1.  9.  0.  0.]
 [ 0.  9.  9.  9. 10.  9.  9.  9.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]


In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(rewards)
plt.xlabel("Episode", fontsize=14)
plt.ylabel("Sum of rewards", fontsize=14)
#save_fig("dqn_rewards_plot")
plt.show()

In [ ]:
#model.set_weights(best_weights)

model.save("/content/drive/MyDrive/Colab Notebooks/Snowman/dddqn1.h5")
prova()

In [ ]:
print(best_score)
